# **LP4_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))



### **Generate Mathematical Model**

In [4]:
problem = """ Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets.

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs)."""

In [10]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [11]:
print(response1.choices[0].message.content)

Parameters:
W1 = Wheat yield per acre = 2.5 T
P1 = Cost to plant an acre of wheat = $150
C1 = Selling price per ton of wheat = $170
D1 = Buying price per ton of wheat = 0.4 * C1 

C2 = Corn yield per acre = 3 T
P2 = Cost to plant an acre of corn = $230
C2 = Selling price per ton of corn = $150
D2 = Buying price per ton of corn = 0.4 * C2 

S = Sugar beet yield per acre = 20 T
PS = Cost to plant an acre of Sugar beet = $260
CS = Selling price per ton of Sugar beet = $36
DS = Selling price per ton of Sugar beet (excess quota) = $10
QS = Quota of sugar beett  = 6000 T

A = Total Area = 500 acres
AW = Minimum wheat required = 200 T
AC = Minimum corn required = 240 T

Decision Variables:
x1 = acres of land used for wheat
x2 = acres of land used for corn
x3 = acres of land used for sugar beet

x4 = excess wheat bought
x5 = excess corn bought

Objective Function (Minimize cost):
Minimize Z = P1*x1 + P2*x2 + PS*x3 + D1*x4 + D2*x5 - C1*W1*x1 - C2*C2*x2 - (CS*S*x3 + DS*(S*x3 - QS)*[S*x3 > QS ])


### **Generate Pyomo Code**

In [12]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [13]:
Markdown(response2.choices[0].message.content)

Here is a Python script utilizing the Pyomo module to represent the given optimization problem:

```python
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define Sets
# N/A

# Define Parameters

# As given in the problem statement
W1 = 2.5
P1 = 150
C1 = 170
D1 = 0.4 * C1 

C2 = 3
P2 = 230
C2 = 150
D2 = 0.4 * C2

S = 20
PS = 260
CS = 36
DS = 10
QS = 6000

A = 500
AW = 200
AC = 240

# Define Variables
model.x1 = Var(domain=NonNegativeReals)  # acres of land used for wheat
model.x2 = Var(domain=NonNegativeReals)  # acres of land used for corn
model.x3 = Var(domain=NonNegativeReals)  # acres of land used for sugar beet
model.x4 = Var(domain=NonNegativeReals)  # excess wheat bought
model.x5 = Var(domain=NonNegativeReals)  # excess corn bought

# Define Objective
def obj_expression(model):
    return P1*model.x1 + P2*model.x2 + PS*model.x3 + D1*model.x4 + D2*model.x5 - C1*W1*model.x1 - C2*C2*model.x2 - (CS*S*model.x3 + DS*(S*model.x3 - QS))

model.OBJ = Objective(rule=obj_expression, sense=minimize)

# Define Constraints
def land_constraint_rule(model):
    return model.x1 + model.x2 + model.x3 <= A

model.Land_Constraint = Constraint(rule=land_constraint_rule)

def wheat_constraint_rule(model):
    return W1*model.x1 - model.x4 >= AW

model.Wheat_Constraint = Constraint(rule=wheat_constraint_rule)

def corn_constraint_rule(model):
    return C2*model.x2 - model.x5 >= AC

model.Corn_Constraint = Constraint(rule=corn_constraint_rule)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)
```

In this script, the `ConcreteModel()` function is used to define the model. The `Var()` function is used to define the decision variables, and the `Objective()` function is used to define the objective function. The `Constraint()` function is used to define the constraints. The GLPK solver is then used to solve the model.  Note that the variables have been restricted to non-negative reals as per the optimization problem specifications.

### **Run the code Generated by GPT4**

In [28]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define Sets
# N/A

# Define Parameters

# As given in the problem statement
W1 = 2.5
P1 = 150
C1 = 170
D1 = 0.4 * C1

C2 = 3
P2 = 230
C2 = 150
D2 = 0.4 * C2

S = 20
PS = 260
CS = 36
DS = 10
QS = 6000

A = 500
AW = 200
AC = 240

# Define Variables
model.x1 = Var(domain=NonNegativeReals)  # acres of land used for wheat
model.x2 = Var(domain=NonNegativeReals)  # acres of land used for corn
model.x3 = Var(domain=NonNegativeReals)  # acres of land used for sugar beet
model.x4 = Var(domain=NonNegativeReals)  # excess wheat bought
model.x5 = Var(domain=NonNegativeReals)  # excess corn bought

# Define Objective
def obj_expression(model):
    return P1*model.x1 + P2*model.x2 + PS*model.x3 + D1*model.x4 + D2*model.x5 - C1*W1*model.x1 - C2*C2*model.x2 - (CS*S*model.x3 + DS*(S*model.x3 - QS))

model.OBJ = Objective(rule=obj_expression, sense=minimize)

# Define Constraints
def land_constraint_rule(model):
    return model.x1 + model.x2 + model.x3 <= A

model.Land_Constraint = Constraint(rule=land_constraint_rule)

def wheat_constraint_rule(model):
    return W1*model.x1 - model.x4 >= AW

model.Wheat_Constraint = Constraint(rule=wheat_constraint_rule)

def corn_constraint_rule(model):
    return C2*model.x2 - model.x5 >= AC

model.Corn_Constraint = Constraint(rule=corn_constraint_rule)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)
print(model.OBJ())

-9315400.0


### **Edit and Run the code for the mathematical model produced by GPT4 (if translation of model into code was incorrect or unexecutable)**